In [1]:
# Load environment variables from a .env file
from dotenv import load_dotenv
import os

load_dotenv(override=True)
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate

In [10]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",
    api_key=GOOGLE_API_KEY,
)

In [11]:
import operator
from typing import Annotated

class State(dict):
    text: str
    outputs: Annotated[list, operator.add] # Reducer to make it append-only

In [21]:
# Nodes

def summarize(state: State) -> State:
    prompt = f"Summarize in one sentence:\n\n{state["text"]}"
    response = llm.invoke(prompt)
    return {"outputs": [f"Summary: {response.content.strip()}"]}

def critique(state: State) -> State:
    prompt = f"Critique briefly:\n\n{state["text"]}"
    response = llm.invoke(prompt)
    return {"outputs": [f"Critique: {response.content.strip()}"]}

def combine(state: State) -> State:
    prompt = f"Combine the following critique and summarisation into one paragraph:\n\n{state['outputs']}"
    response = llm.invoke(prompt)
    return {"outputs": [f"Combined:\n{response.content.strip()}"]}

In [22]:
from langgraph.graph import StateGraph, START, END

# Build the graph
workflow = StateGraph(State)
workflow.add_node("summarize", summarize)
workflow.add_node("critique", critique)
workflow.add_node("combine", combine)

workflow.add_edge(START, "summarize")
workflow.add_edge(START, "critique")

workflow.add_edge("summarize", "combine")
workflow.add_edge("critique", "combine")

workflow.add_edge("combine", END)

In [23]:
# Compile graph
graph = workflow.compile()

In [24]:
# TEST
input_text = "World hunger is a solvable problem. Basic maths can be used to verify that redistribution of wealth can bring all population of the world out of poverty."

result = graph.invoke({"text": input_text, "outputs": []})

print("Final outputs:")
for item in result["outputs"]:
    print(item)
    print("=" * 75)

Final outputs:
Critique: Here's a brief critique of the statement:

While the sentiment that world hunger is solvable is widely shared, the claim that "basic maths" and "redistribution of wealth" are a straightforward solution is an oversimplification.

**Critique Points:**

*   **Oversimplification of "Basic Maths":** While aggregate statistics might suggest enough food exists globally, solving hunger involves far more than just numbers. It requires understanding complex logistical chains, infrastructure, agricultural yields, storage, waste, and distribution networks, all of which are far from simple.
*   **Oversimplification of "Redistribution of Wealth":** Wealth redistribution is an incredibly complex political, economic, and social undertaking. It involves issues of property rights, incentives for production, governance, corruption, and varying economic systems. Simply stating that it *can* be done ignores the immense practical and ideological hurdles.
*   **Ignores Root Causes:**